Demonstration of the transitive path cache.

This cache stores "`a -> b`" transitions separate from "`a -> a`" self-transitions. 

This is appropriate to store lambda calculus reductions. Reductions being the subset of lambda calculus beta-transformations that terminate (don't grow forever or cycle).

For this cache a lookup keyed by "`a`" returns the current end of path for "`a`" and caches the "`a`" to end of path result. If a cycle is detected during lookup a `ValueError()` is raised. Cycles are not detected until lookup, which is where the transitive closure of the path is detected.

In [1]:
from lambda_calc import *
from TransitiveCache import TransitiveCache

In [2]:
tc = TransitiveCache()

In [3]:
tc.store_transition("a", "b")

In [4]:
tc.store_transition("b", "c")

In [5]:
tc._transitions

OrderedDict([('a', 'b'), ('b', 'c')])

Lookup where "`a`" ends.

In [6]:
tc.lookup_result("a")

'c'

The final destination of "`a`" is cached after lookup.

In [7]:
tc._transitions

OrderedDict([('b', 'c'), ('a', 'c')])

Force a cycle.

In [8]:
tc.store_transition("c", "a")

Cycle is detected at lookup.

In [9]:
try:
    tc.lookup_result("a")
except ValueError as ve:
    print(f"saw ValueError '{ve}'")

saw ValueError 'cycle'


We show how to use this with the blinker lambda expression.

In [10]:
a = λ["x", "y"]("y", "x", "y")
blinker = a | a | a

blinker

v((λ['x'](λ['y'](('y', 'x'), 'y')), λ['x'](λ['y'](('y', 'x'), 'y'))), λ['x'](λ['y'](('y', 'x'), 'y')))

The blinker doesn't have a normal form. It keeps changing back and forth.

In [11]:
b_1 = blinker.r()
assert blinker != b_1

b_1

v(λ['y'](('y', λ['x'](λ['y'](('y', 'x'), 'y'))), 'y'), λ['x'](λ['y'](('y', 'x'), 'y')))

In [12]:
b_2 = b_1.r()
assert b_1 != b_2
assert b_2 == blinker

b_2

v((λ['x'](λ['y'](('y', 'x'), 'y')), λ['x'](λ['y'](('y', 'x'), 'y'))), λ['x'](λ['y'](('y', 'x'), 'y')))

This is why both the "`.nf()`" normal form calculation and the cache require cycle detection. Else either of those would spin forever on oscillating reductions. Though, this does not prevent forever runs for expressions that are not repeating.

In [13]:
tc = TransitiveCache()
tc.store_transition(blinker, b_1)
tc.store_transition(b_1, b_2)
tc._transitions


OrderedDict([(v((λ['x'](λ['y'](('y', 'x'), 'y')), λ['x'](λ['y'](('y', 'x'), 'y'))), λ['x'](λ['y'](('y', 'x'), 'y'))),
              v(λ['y'](('y', λ['x'](λ['y'](('y', 'x'), 'y'))), 'y'), λ['x'](λ['y'](('y', 'x'), 'y')))),
             (v(λ['y'](('y', λ['x'](λ['y'](('y', 'x'), 'y'))), 'y'), λ['x'](λ['y'](('y', 'x'), 'y'))),
              v((λ['x'](λ['y'](('y', 'x'), 'y')), λ['x'](λ['y'](('y', 'x'), 'y'))), λ['x'](λ['y'](('y', 'x'), 'y'))))])

In [14]:
try:
    tc.lookup_result(blinker)
except ValueError as ve:
    print(f"saw ValueError '{ve}'")

saw ValueError 'cycle'
